In [43]:
import numpy as np
import pandas as pd
import requests

In [2]:
# Parse PIF summary Excel file into pandas dataframe
pif_summary_xlsx = '/Users/lchambers/TEL/Commissioning/PIF Status Summary 2019-01.xlsx'
pif_summary = pd.read_excel(pif_summary_xlsx)
pif_summary.columns

In [19]:
# Look at columns for OTE # and APT program #
pif_summary[['OTE #', 'APT Prog. ID']]

,OTE #,APT Prog. ID
0,OTE-00,NaN
1,NaN,NaN
2,NRC-01 to 03,NaN
3,OTE-01,1134
4,OTE-02,1135
5,OTE-03,1136
6,OTE-04,1137
7,OTE-05,1138
8,MU,NaN
9,OTE-06,1140


In [37]:
# Start to build commissioning dictionary
commissioning_dict = {}
for index, row in pif_summary.iterrows():
    car = row['OTE #']
    apt = row['APT Prog. ID']
#     print(row['APT Prog. ID'], type(row['APT Prog. ID']), pd.isna(row['APT Prog. ID']))
    if pd.notna(car) and pd.notna(apt) and \
        isinstance(apt, int) and len(car.split()) == 1:
        
        commissioning_dict[car.lower()] = {}
        commissioning_dict[car.lower()]['apt'] = apt
        
#         print(row['OTE #'], row['APT Prog. ID'], type(row['APT Prog. ID']))

commissioning_dict

{'ote-01': {'apt': 1134},
 'ote-02': {'apt': 1135},
 'ote-03': {'apt': 1136},
 'ote-04': {'apt': 1137},
 'ote-05': {'apt': 1138},
 'ote-06': {'apt': 1140},
 'los-2': {'apt': 1410},
 'ote-07': {'apt': 1141},
 'ote-08': {'apt': 1142},
 'ote-09': {'apt': 1143},
 'ote-10': {'apt': 1144},
 'ote-11': {'apt': 1145},
 'ote-12': {'apt': 1147},
 'ote-13': {'apt': 1148},
 'ote-14': {'apt': 1149},
 'ote-15': {'apt': 1150},
 'ote-16': {'apt': 1151},
 'ote-17': {'apt': 1153},
 'ote-18': {'apt': 1155},
 'ote-19': {'apt': 1156},
 'ote-21': {'apt': 1158},
 'ote-22': {'apt': 1159},
 'ote-23': {'apt': 1160},
 'ote-25': {'apt': 1162},
 'ote-27': {'apt': 1164},
 'ote-26': {'apt': 1163},
 'ote-28': {'apt': 1165},
 'ote-29': {'apt': 1166},
 'ote-31': {'apt': 1168},
 'ote-32': {'apt': 1169},
 'ote-24': {'apt': 1161},
 'ote-34': {'apt': 1171},
 'ote-33': {'apt': 1170},
 'ote-35': {'apt': 1172}}

In [42]:
!mkdir temp_apt

!rm -r temp_apt

mkdir: temp_apt: File exists


In [40]:
# Open up each APT file enough to get the number of observations...

! /Applications/APT\ 26.2.2/bin/apt -nogui -export pointing,xml www.stsci.edu/jwst/phase2-public/1148.aptx

Starting APT...
------------------------
Initial Locale:  en_US = English (United States)
------------------------
Overriding initial Locale with:  en_US = English (United States)
------------------------
System Properties:
	allow.submission               : true
	apt.debug                      : false
	awt.toolkit                    : sun.lwawt.macosx.LWCToolkit
	file.encoding                  : UTF8
	file.encoding.pkg              : sun.io
	file.separator                 : /
	ftp.nonProxyHosts              : local|*.local|169.254/16|*.169.254/16
	gopherProxySet                 : false
	http.agent                     : apt/26.2.2 (Mac OS X 10.12.6; U; JRE 10.0.1)
	http.nonProxyHosts             : local|*.local|169.254/16|*.169.254/16
	java.awt.graphicsenv           : sun.awt.CGraphicsEnvironment
	java.awt.headless              : true
	java.awt.printerjob            : sun.lwawt.macosx.CPrinterJob
	java.class.path                : /Applications/APT 26.2.2/bin/../APT.app/Contents/Java/APT